In [1]:
from numpy.core.numeric import True_
import sys
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.linear_model import LogisticRegression
#from matplotlib import pyplot as plt
#from PIL import Image, ImageDraw
#from IPython import display
#from torchvision import transforms

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [3]:
data = 'lending_club'
df = pd.read_csv('../datasets/'+data+'_sc.csv')

In [4]:
df.head()

,emp_length,annual_inc,open_acc,cr_history_yrs,grade,home_ownership,purpose,addr_state,loan_status
0,1.0,0.003336,0.023810,0.380952,0.879607,0.847922,0.892625,0.855825,1
1,0.1,0.004336,0.023810,0.158730,0.830560,0.847922,0.895807,0.844324,0
2,1.0,0.001376,0.000000,0.111111,0.830560,0.847922,0.735034,0.869212,1
3,1.0,0.007538,0.190476,0.206349,0.830560,0.847922,0.838714,0.839524,1
4,0.3,0.005337,0.166667,0.063492,0.940174,0.847922,0.897936,0.855825,1


In [5]:
target = 'loan_status'
y = df[target].values
del df[target]
x = df.values

In [6]:
# train the classifier to be explained -> Logistic Regression Model (can be replaced with a MLP)
# clf = LogisticRegression(max_iter=1000, fit_intercept=False, class_weight='balanced')
# clf.fit(x, y)

In [7]:
# Convert numpy arrays to PyTorch tensors
x_train_tensor = torch.FloatTensor(x).to(device)
y_train_tensor = torch.LongTensor(y).to(device)

In [8]:
# Create a TensorDataset
dataset = TensorDataset(x_train_tensor, y_train_tensor)

In [9]:
# Create a DataLoader
dl_batch_size = 32  # Set your desired batch size
train_dataloader = DataLoader(dataset, batch_size=dl_batch_size, shuffle=True)

In [10]:
num_epochs = 500

In [11]:
x.shape[1]

8

In [12]:
latent_dims = 4
# capacity = no. of hidden neurons in a layer
capacity = 8
variational_beta = 1

In [13]:
latent_dims

4

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim):
        super(Encoder, self).__init__()
        c = capacity
        self.fc1 = nn.Linear(in_features=input_dim, out_features=c*2)
        self.fc2 = nn.Linear(in_features=c*2, out_features=latent_dims)
        self.fc3 = nn.Linear(in_features=c*2, out_features=latent_dims)
            
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc2(x)
        x_logvar = self.fc3(x)
        return x_mu, x_logvar

class Decoder(nn.Module):
    def __init__(self, output_dim):
        super(Decoder, self).__init__()
        c = capacity
        self.fc = nn.Linear(in_features=latent_dims, out_features=c*2)
        self.fc_output = nn.Linear(in_features=c*2, out_features=output_dim)
            
    def forward(self, x):
        x = F.relu(self.fc(x))
        x_recon = self.fc_output(x)
        return x_recon

class VariationalAutoencoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder(input_dim)
        self.decoder = Decoder(output_dim)
    
    def forward(self, x):
        latent_mu, latent_logvar = self.encoder(x)
        latent = self.latent_sample(latent_mu, latent_logvar)
        x_recon = self.decoder(latent)
        return x_recon, latent_mu, latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')  # Mean Squared Error loss for tabular data
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + variational_beta * kldivergence

In [15]:
# Train VAE
print(device)
vae = VariationalAutoencoder(input_dim=x.shape[1], output_dim=x.shape[1])
vae = vae.to(device)

optimizer = torch.optim.Adam(params=vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_vae = True

if train_vae == False:   
    filename = 'blood_alcohol_vae.pth'
    vae.load_state_dict(torch.load(filename))
    vae.eval()
    print('done')

else:
    for epoch in range(num_epochs):
        vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for x_batch, _ in train_dataloader:

            x_batch = x_batch.to(device)

            # Forward pass
            x_recon, latent_mu, latent_logvar = vae(x_batch)
            loss = vae_loss(x_recon, x_batch, latent_mu, latent_logvar)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average loss: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_vae.pth'
    torch.save(vae.state_dict(), model_name)

mps
Epoch [1 / 500] average loss: 13.616909
Epoch [2 / 500] average loss: 4.886917
Epoch [3 / 500] average loss: 4.809705
Epoch [4 / 500] average loss: 4.796064
Epoch [5 / 500] average loss: 4.791015
Epoch [6 / 500] average loss: 4.787696
Epoch [7 / 500] average loss: 4.789276
Epoch [8 / 500] average loss: 4.788477
Epoch [9 / 500] average loss: 4.786935
Epoch [10 / 500] average loss: 4.784735
Epoch [11 / 500] average loss: 4.784076
Epoch [12 / 500] average loss: 4.786365
Epoch [13 / 500] average loss: 4.783971
Epoch [14 / 500] average loss: 4.781982
Epoch [15 / 500] average loss: 4.780855
Epoch [16 / 500] average loss: 4.782954
Epoch [17 / 500] average loss: 4.780494
Epoch [18 / 500] average loss: 4.782536
Epoch [19 / 500] average loss: 4.780638
Epoch [20 / 500] average loss: 4.781973
Epoch [21 / 500] average loss: 4.780918
Epoch [22 / 500] average loss: 4.779933
Epoch [23 / 500] average loss: 4.779534
Epoch [24 / 500] average loss: 4.778000
Epoch [25 / 500] average loss: 4.780521
Epoc

Epoch [204 / 500] average loss: 4.770951
Epoch [205 / 500] average loss: 4.772274
Epoch [206 / 500] average loss: 4.772211
Epoch [207 / 500] average loss: 4.772409
Epoch [208 / 500] average loss: 4.772063
Epoch [209 / 500] average loss: 4.770147
Epoch [210 / 500] average loss: 4.772277
Epoch [211 / 500] average loss: 4.772048
Epoch [212 / 500] average loss: 4.773005
Epoch [213 / 500] average loss: 4.772594
Epoch [214 / 500] average loss: 4.772635
Epoch [215 / 500] average loss: 4.772282
Epoch [216 / 500] average loss: 4.772704
Epoch [217 / 500] average loss: 4.772002
Epoch [218 / 500] average loss: 4.770304
Epoch [219 / 500] average loss: 4.771605
Epoch [220 / 500] average loss: 4.771615
Epoch [221 / 500] average loss: 4.769746
Epoch [222 / 500] average loss: 4.771548
Epoch [223 / 500] average loss: 4.771935
Epoch [224 / 500] average loss: 4.771517
Epoch [225 / 500] average loss: 4.771604
Epoch [226 / 500] average loss: 4.771268
Epoch [227 / 500] average loss: 4.772388
Epoch [228 / 500

Epoch [404 / 500] average loss: 4.770159
Epoch [405 / 500] average loss: 4.769549
Epoch [406 / 500] average loss: 4.769868
Epoch [407 / 500] average loss: 4.770906
Epoch [408 / 500] average loss: 4.770345
Epoch [409 / 500] average loss: 4.770046
Epoch [410 / 500] average loss: 4.769964
Epoch [411 / 500] average loss: 4.770198
Epoch [412 / 500] average loss: 4.769908
Epoch [413 / 500] average loss: 4.769877
Epoch [414 / 500] average loss: 4.769919
Epoch [415 / 500] average loss: 4.769934
Epoch [416 / 500] average loss: 4.770409
Epoch [417 / 500] average loss: 4.770200
Epoch [418 / 500] average loss: 4.769620
Epoch [419 / 500] average loss: 4.770405
Epoch [420 / 500] average loss: 4.769424
Epoch [421 / 500] average loss: 4.769390
Epoch [422 / 500] average loss: 4.769930
Epoch [423 / 500] average loss: 4.769852
Epoch [424 / 500] average loss: 4.769854
Epoch [425 / 500] average loss: 4.769339
Epoch [426 / 500] average loss: 4.770428
Epoch [427 / 500] average loss: 4.770042
Epoch [428 / 500

In [16]:
c2c_latent_dims = latent_dims-1
print(c2c_latent_dims)
c2c_capacity = latent_dims
print(c2c_capacity)
c2c_variational_beta = 1
label_size = 2  # Assuming two classes

3
4


In [17]:
class c2c_Encoder(nn.Module):
    def __init__(self):
        super(c2c_Encoder, self).__init__()
        self.fc1 = nn.Linear(in_features=latent_dims + label_size, out_features=c2c_capacity)
        self.fc_mu = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
        self.fc_logvar = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc_mu(x)
        x_logvar = self.fc_logvar(x)
        return x_mu, x_logvar

class c2c_Decoder(nn.Module):
    def __init__(self):
        super(c2c_Decoder, self).__init__()
        self.fc2 = nn.Linear(in_features=c2c_latent_dims + label_size, out_features=c2c_capacity)
        self.fc1 = nn.Linear(in_features=c2c_capacity, out_features=latent_dims)
            
    def forward(self, x):
        x = self.fc2(x)
        x = self.fc1(x)
        return x
    
class c2c_VariationalAutoencoder(nn.Module):
    def __init__(self):
        super(c2c_VariationalAutoencoder, self).__init__()
        self.encoder = c2c_Encoder()
        self.decoder = c2c_Decoder()
    
    def forward(self, x):
        x,label = torch.split(x,(latent_dims,label_size),dim = 1)
        c2c_latent_mu, c2c_latent_logvar = self.encoder(torch.cat((x,label), dim=1))
        c2c_latent = self.latent_sample(c2c_latent_mu, c2c_latent_logvar)
        x_recon = self.decoder(torch.cat((c2c_latent,label), dim=1))
        return x_recon, c2c_latent_mu, c2c_latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def c2c_vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x.view(-1, latent_dims), x.view(-1, latent_dims), reduction='sum')
    
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + c2c_variational_beta * kldivergence

In [18]:
print(device)
c2c_vae = c2c_VariationalAutoencoder()
c2c_vae = c2c_vae.to(device)

optimizer = torch.optim.Adam(params=c2c_vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_c2c_vae = True

if train_c2c_vae == False:
    filename = 'blood_alcohol_c2c.pth'
    c2c_vae.load_state_dict(torch.load(filename))
    c2c_vae.eval()
    print('done')
else:
    for epoch in range(num_epochs):
        c2c_vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for i in range(dl_batch_size):

            x_batch1, labels1 = next(iter(train_dataloader))
            x_batch2, labels2 = next(iter(train_dataloader))

            x_batch1 = x_batch1.to(device)
            x_batch_recon1, latent_mu1, latent_logvar1 = vae(x_batch1)

            x_batch2 = x_batch2.to(device)
            x_batch_recon2, latent_mu2, latent_logvar2 = vae(x_batch2) 

            latent_diff = latent_mu1 - latent_mu2 
            label_diff = torch.cat((labels1.unsqueeze(1), labels2.unsqueeze(1)), dim=1)
            #print(label_diff)

            latent_diff = latent_diff.to(device)
            label_diff = label_diff.float().to(device)

            c2c_input = torch.cat((latent_diff,label_diff), dim=1)
            c2c_input = c2c_input.to(device)

            c2c_recon, c2c_latent_mu, c2c_latent_logvar = c2c_vae(c2c_input)
            loss = c2c_vae_loss(c2c_recon, latent_diff.detach(), c2c_latent_mu, c2c_latent_logvar)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average reconstruction error: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_c2c.pth'
    torch.save(c2c_vae.state_dict(), model_name)

mps
Epoch [1 / 500] average reconstruction error: 46.448065
Epoch [2 / 500] average reconstruction error: 31.141906
Epoch [3 / 500] average reconstruction error: 22.062232
Epoch [4 / 500] average reconstruction error: 16.345660
Epoch [5 / 500] average reconstruction error: 12.088543
Epoch [6 / 500] average reconstruction error: 9.156775
Epoch [7 / 500] average reconstruction error: 6.937803
Epoch [8 / 500] average reconstruction error: 5.318635
Epoch [9 / 500] average reconstruction error: 4.044519
Epoch [10 / 500] average reconstruction error: 3.032776
Epoch [11 / 500] average reconstruction error: 2.358679
Epoch [12 / 500] average reconstruction error: 1.774963
Epoch [13 / 500] average reconstruction error: 1.359427
Epoch [14 / 500] average reconstruction error: 1.145081
Epoch [15 / 500] average reconstruction error: 0.904428
Epoch [16 / 500] average reconstruction error: 0.752974
Epoch [17 / 500] average reconstruction error: 0.577790
Epoch [18 / 500] average reconstruction error: 0

Epoch [147 / 500] average reconstruction error: 0.000071
Epoch [148 / 500] average reconstruction error: 0.000067
Epoch [149 / 500] average reconstruction error: 0.000054
Epoch [150 / 500] average reconstruction error: 0.000050
Epoch [151 / 500] average reconstruction error: 0.000044
Epoch [152 / 500] average reconstruction error: 0.000038
Epoch [153 / 500] average reconstruction error: 0.000033
Epoch [154 / 500] average reconstruction error: 0.000030
Epoch [155 / 500] average reconstruction error: 0.000027
Epoch [156 / 500] average reconstruction error: 0.000023
Epoch [157 / 500] average reconstruction error: 0.000019
Epoch [158 / 500] average reconstruction error: 0.000017
Epoch [159 / 500] average reconstruction error: 0.000014
Epoch [160 / 500] average reconstruction error: 0.000014
Epoch [161 / 500] average reconstruction error: 0.000011
Epoch [162 / 500] average reconstruction error: 0.000010
Epoch [163 / 500] average reconstruction error: 0.000008
Epoch [164 / 500] average recon

Epoch [289 / 500] average reconstruction error: -0.000002
Epoch [290 / 500] average reconstruction error: -0.000002
Epoch [291 / 500] average reconstruction error: -0.000002
Epoch [292 / 500] average reconstruction error: -0.000002
Epoch [293 / 500] average reconstruction error: -0.000002
Epoch [294 / 500] average reconstruction error: -0.000002
Epoch [295 / 500] average reconstruction error: -0.000002
Epoch [296 / 500] average reconstruction error: -0.000002
Epoch [297 / 500] average reconstruction error: -0.000001
Epoch [298 / 500] average reconstruction error: -0.000002
Epoch [299 / 500] average reconstruction error: -0.000002
Epoch [300 / 500] average reconstruction error: -0.000002
Epoch [301 / 500] average reconstruction error: -0.000002
Epoch [302 / 500] average reconstruction error: -0.000002
Epoch [303 / 500] average reconstruction error: -0.000001
Epoch [304 / 500] average reconstruction error: -0.000002
Epoch [305 / 500] average reconstruction error: -0.000002
Epoch [306 / 5

Epoch [431 / 500] average reconstruction error: -0.000003
Epoch [432 / 500] average reconstruction error: -0.000002
Epoch [433 / 500] average reconstruction error: -0.000002
Epoch [434 / 500] average reconstruction error: -0.000002
Epoch [435 / 500] average reconstruction error: -0.000002
Epoch [436 / 500] average reconstruction error: -0.000002
Epoch [437 / 500] average reconstruction error: -0.000000
Epoch [438 / 500] average reconstruction error: -0.000002
Epoch [439 / 500] average reconstruction error: -0.000001
Epoch [440 / 500] average reconstruction error: 0.000009
Epoch [441 / 500] average reconstruction error: 0.000007
Epoch [442 / 500] average reconstruction error: 0.000008
Epoch [443 / 500] average reconstruction error: 0.000078
Epoch [444 / 500] average reconstruction error: 0.000104
Epoch [445 / 500] average reconstruction error: 0.000067
Epoch [446 / 500] average reconstruction error: 0.000021
Epoch [447 / 500] average reconstruction error: 0.000004
Epoch [448 / 500] aver